# 1 Dimensional Example

$dY_{t} =  -f(Y) \,dt + g(Y)\, dW_{t}$

In [ ]:
from SDEsim import SDE

In [ ]:
#help(SDE)
#Declaring a simulation with 40,000,000 parallel solvers
sim = SDE(40_000_000, dtype = 'float64', method = 'milstein', device = 'cuda', tolerance=1e-4)

### $dY_{t} =  -Y \,dt + 0.1\,dW_{t}$

In [ ]:
#Inputting simulation equation
f = lambda y: -y #drift
g = lambda y: 0.1 #diffusion
dg = lambda y: 0.0 #derivative of g, not required when method = 'euler_maruyama'
df = lambda y: -1.0 #derivative of drift, it is required for measuring information rate of velocity or if using methods with higher order than milstein
#sim.debug=False
sim.set_functions(f, g, df, dg) #if dg or df not given it will automatically be assumed to be 0.
#Setting up stats measurements, with collection_frequency, once every 2 time steps
sim.set_stats(info_rate=True, info_rate_v=True, density= True, density_v=False, mean_v=False, std_v=False, mean=True, std=True, entropy=False,entropy_v=False,collection_frequency=2)

In [ ]:
#run simulation
sim.simulate(t_init=0, t_end=100, init_mean=100, init_std=0.0001)
#save data
sim.save_data(output_directory = r'data', filename = f'linear.dic')

### $dY_{t} =  -\left(Y^3 - 2Y\right)\,dt+0.8\,dW_{t}$

In [ ]:
f = lambda y: -y**3 + 2*y #drift
g = lambda y: 0.8 #diffusion
dg = lambda y: 0.0 #derivative of g, not required when method = 'euler_maruyama'
df = lambda y: -3*y**2 + 2 #derivative of drift, it is required for measuring information rate of velocity or if using methods with higher order than milstein
sim.set_functions(f, g, df,dg)

sim.simulate(t_init=0, t_end=100, init_mean=0.0, init_std=0.5)
sim.save_data(output_directory = r'data', filename = f'non_linear.dic')

# N Dimensional Example

$
\partial_{t} \mathcal{E}=\mathcal{E} \mathcal{N}-a_{1} \mathcal{E}^{2}-a_{2} V^{2} \mathcal{E}-a_{3} V_{\mathrm{ZF}}^{2} \mathcal{E} \\
\partial_{t} V_{\mathrm{ZF}}=b_{1} \frac{\mathcal{E} V_{\mathrm{ZF}}}{1+b_{2} V^{2}}-b_{3} V_{\mathrm{ZF}} \\
\partial_{t} \mathcal{N}=-c_{1} \mathcal{E} \mathcal{N}-c_{2} \mathcal{N}+t_c \, t + 0.001\,dW_t
$

<br>
These equations are stiff. Numerical solution requires very small tolerance. This will cause significant slow down.
This method is only advantageus when running large number of parallel simulations (order of millions) with CUDA.

In [ ]:
from SDEsim import SDE_ND

In [ ]:
sim = SDE_ND(2_000_000, dims = 3, dtype='float64', device = 'cuda', method='milstein', dt_init=1e-10)

In [ ]:
sim.debug=False

sim.tolerance = 1e-9 #very low tolerance because of stiffness in the equation
sim.facmax = 1.05 #the adaptive step size will increase by a maximum factor of 1.05

a1, a2, a3 = 0.2, 0.7, 0.7
b1, b2, b3 = 1.5, 1, 1
c1, c2, d = 1, 0.5, 1
tc = 0.01

f0 = f'y0*y2 - {a1}*y0**2 - {a2*d**2}*y2**4*y0 - {a3}*y1**2*y0'
f1 = f'{b1}*y0*y1/(1+{b2*d**2}*y2**4) - {b3}*y1'
f2 = f'-{c1}*y0*y2 - {c2}*y2 + {tc}*t + 0.1'

g0 = 0
g1 = 0
g2 = lambda x: 1e-3

#dg (derivative of g) is 0 for all cases so not given
sim.set_functions((f0,g0),(f1,g1),(f2,g2))
#collection_frequency=10, collects stats every 10 time steps.
sim.set_stats(mean_v = True, std_v = True, mean=True, std=True, entropy=True, density=True, entropy_v=True, collection_frequency=10)


In [ ]:
sim.simulate(t_init=0, t_end=200, init_mean=0.01, init_std=0.0)
sim.save_data(output_directory = r'data', filename = f'ND.dic')

### Plotting

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
x = sim.data['t'][::]*0.01
eqs = [r'$\mathcal{\epsilon}$', r'$V_{ZF}$', r'$\mathcal{N}/5$']

for i in range(3):
    y = sim.data['mean(t)'][::,i]
    y_std = sim.data['std(t)'][::,i]*10
    if i==2: y=y/5; y_std=y_std/5
        
    plt.plot(x, y, label = eqs[i])
    plt.fill_between(x, y-y_std, y+y_std, alpha=0.3)
    
#x = temp.data['t'][::]*0.01
eqs = ['1e-5::'+i for i in eqs]

plt.xlabel('Q')
plt.ylabel('y')
plt.legend()
#plt.yscale('log')
plt.show()